In [ ]:
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import StatevectorSampler as Sampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli
import numpy as np
from qiskit.circuit.library import XXPlusYYGate
import qiskit

In [ ]:


def build_cost_layer(qc, h, w, gamma):
    """Apply e^{-i * gamma * H_cost}"""
    n = len(h)
    # Linear terms: h_i Z_i
    for i in range(n):
        if h[i] != 0:
            qc.rz(2 * gamma * h[i], i)


def build_mixer_layer(qc, beta):
    """Apply e^{-i * beta * H_mix} where H_mix = sum X_i"""
    # print(2*np.pi,beta)
    theta = 2 * np.pi * beta
    for i in range(qc.num_qubits - 1):
        qc.append(XXPlusYYGate(theta), [qc.qubits[i], qc.qubits[i+1]])

    # for q in range(qc.num_qubits):
    #     qc.rx(2 * beta, q)
        # perhaps use RXY SWap gate?


def qaoa_run(h, w, gammas, betas):
    """
    h: list of linear coefficients
    w: dict with keys (i,j) for quadratic coefficients
    gammas, betas: lists of length p
    """
    p = len(gammas)
    n = len(h)

    # Step 1: initialize |+>^n
    qc = QuantumCircuit(n)
    qc.h(range(n))

    # Step 2: QAOA layers
    for gamma, beta in zip(gammas, betas):
        build_cost_layer(qc, h, w, gamma)
        build_mixer_layer(qc, beta)

    qc.measure_all()

    # Step 3: run on simulator
    simulator = AerSimulator()
    # result = execute(qc, backend, shots=2048).result()
    basis_gate = ['id', 'rz', 'sx', 'x', 'cx', 'reset', 'measure']
    # print(simulator.configuration().basis_gates)
    qc_t = qiskit.transpile(
    qc, 
    simulator,
    basis_gates=['rz', 'sx', 'x', 'cx'],
    optimization_level=3
    )

    result_ideal = simulator.run(qc_t, shots=10_000).result()    
    counts = result_ideal.get_counts()
    print(counts)

    # Step 4: return optimal bitstring (max probability)
    best_bitstring = max(counts, key=counts.get)
    return best_bitstring, counts


In [ ]:

def qaoa_with_angles(qp: QuadraticProgram, gammas, betas):
    p = len(gammas)
    reps = p

    # Put all angles into the single array expected by QAOA
    # Order: [γ1,...,γp, β1,...,βp]
    params = gammas + betas

    qaoa = QAOA(sampler=Sampler(), reps=reps, initial_point=params)
    solver = MinimumEigenOptimizer(qaoa)

    result = solver.solve(qp)
    return result.x  # optimal bitstring


In [ ]:
# Example: MaxCut on a 3‑node line graph
h = [0, 0, 0]
w = {(0,1): 1, (1,2): 1}

gammas = [0.8, 1.2]
betas  = [0.4, 0.9]

T = 24
h = np.random.randint(0,2,size = T)

best, counts = qaoa_run(h, w, gammas, betas)
print("Best bitstring:", best)


{'101': 1046, '011': 1200, '111': 1240, '001': 1540, '110': 1541, '100': 1194, '010': 1003, '000': 1236}
Best bitstring: 110


/Users/yanislefur/Documents/IQM_Hackathon/venv312/lib/python3.12/site-packages/qiskit/compiler/transpiler.py:269: UserWarning: Providing `coupling_map` and/or `basis_gates` along with `backend` is not recommended, as this will invalidate the backend's gate durations and error rates.
  pm = generate_preset_pass_manager(


### Test connection IQM